In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import utils.minimization as minim

In [ ]:
# Reloading modules
importlib.reload(minim)

In [ ]:
x = pd.read_csv('parkinsons_updrs.csv')
features = x.columns  # first columns will contain the names of the features
print(f'Dataset size = {x.shape}')
print(f'Dataset features count = {len(features)}')
# print(features)

Features list:
- subject#: patient number
- age: age of the patient
- sex: sex of the patient
- test_time: period from the beginning of the measurement (integer -> day, float -> hour from 0AM to 12PM)
- motor_UPDRS:
- total_UPDRS:
Jitter, in general, is the variation of the fundamental frequency in signals that should be periodic but are not
- Jitter(%), Jitter(Abs), Jitter:RAP, Jitter:PPQ5, Jitter:DDP
Shimmer, in general, is the variation of amplitude in signals that should be periodic but are not
- Shimmer, Shimmer(dB), Shimmer:APQ3, Shimmer:APQ5, Shipper:APQ11, Shimmer:DDA
- NHR: Noise to Harmonics Ratio
- HNR: Harmonic to Noise Ratio
- RPDE: Recurrence Period Density Entropy
- DFA: Detrended Fluctuation Analysis
- PPE: Perceived Vocal Effort is the measure of effort (physical and cognitive) used to produce speech

In [ ]:
# print(pd.unique(x['subject#']))
# x.plot.scatter('subject#', 'age')
x.loc[:, ~x.columns.isin(['subject#', 'sex'])].plot.hist(
    bins=50)  # 'subject#' and 'sex' are removed to have a clearer histogram
# bins is the number of rectangles, so the lesser -> the more values will be represented with a single rectangle

In [ ]:
X = pd.DataFrame()
subj = np.unique(x['subject#'])
for k in subj:  # for each subject#
    xk = x[x['subject#'] == k]
    xk1 = xk.copy()
    xk1.test_time = xk1.test_time.astype(int)
    xk1['g'] = xk1['test_time']  # new property called 'g' containing the 'test_time' property
    # now the dataframe collapses using 'g' as pivot, and computing the mean for each other property
    v = xk1.groupby('g').mean()
    X = pd.concat([X, v], axis=0, ignore_index=True)  # axis=0 -> concatenation along rows. (0, 1) -> (rows, columns)
features = x.columns
Np, Nc = X.shape
print(f'X shape = {X.shape}')

In [ ]:
Xnorm = (X - X.mean()) / X.std()
c = Xnorm.cov()

In [ ]:
plt.figure()
plt.matshow(np.abs(c.values), fignum=0)
plt.xticks(np.arange(len(features)), features, rotation=90)
plt.yticks(np.arange(len(features)), features, rotation=0)
plt.colorbar()
plt.title('just the title')
plt.tight_layout()
plt.savefig('./corr_coeff.png')
plt.show()

In [ ]:
plt.figure()
c.total_UPDRS.plot()
plt.grid()
plt.xticks(np.arange(len(features)), features, rotation=90)
plt.axhline(y=0, color='g')
plt.title('Correlation coefficients of total_UPDRS versus other features')
plt.tight_layout()
plt.show()

In [ ]:
seed = 319244
Xsh = X.sample(frac=1, replace=False, random_state=seed, axis=0, ignore_index=True)

In [ ]:
# Percentages
percent_training = 0.5
percent_test = 0.25
percent_validation = 1 - percent_training - percent_test

In [ ]:
Ntr = int(Np * percent_training)  # Number of training points
Nte = int(Np * percent_test)  # Number of test points
Nva = Np - Ntr - Nte

X_tr = Xsh[:Ntr]

mm = X_tr.mean()
ss = X_tr.std()
my = mm['total_UPDRS']
sy = ss['total_UPDRS']
mm_vals = mm.values
ss_vals = ss.values

In [49]:
Xsh_norm = (Xsh - mm_vals) / ss_vals  # normalization of all parameters
ysh_norm = Xsh_norm['total_UPDRS']  # total_UPDRS will be the regressand
# Xsh_norm = Xsh_norm[['motor_UPDRS', 'age', 'PPE']]  # keeping only these three regressors
Xsh_norm = Xsh_norm.drop(['total_UPDRS', 'subject#', 'test_time'], axis=1)  # removing the regressand and other useless
# or known correlated features

X_tr_norm = Xsh_norm[:Ntr].values
X_va_norm = Xsh_norm[Ntr:Ntr + Nva].values
X_te_norm = Xsh_norm[-Nte:].values
y_tr_norm = ysh_norm[:Ntr].values
y_va_norm = ysh_norm[Ntr:Ntr + Nva].values
y_te_norm = ysh_norm[-Nte:].values

In [50]:
# Useful functions
def eucl_distance(v1, v2):
    assert len(v1) == len(v2)
    v_diff = v1 - v2
    return pow(v_diff.T @ v_diff, 0.5)

In [51]:
# Solver control plane
r2 = 100
t = 1
s2 = 0.001
N = 10

In [54]:
for row in range(Nva):
    x = X_va_norm[row]
    distances = [eucl_distance(x, X_tr_norm[i]) for i in range(Ntr)]
    i_order = np.argsort(distances)
    closer_X = np.array([X_tr_norm[i] for i in i_order[:N - 1]])
    closer_y = np.array([y_tr_norm[i] for i in i_order[:N - 1]])
    Xr = np.concatenate((closer_X, [x]), axis=0)
    Rn = np.empty(shape=(N, N))
    for n in range(N):
        for k in range(N):
            Rn[n, k] = t * np.exp(- (np.linalg.norm(Xr[n] - Xr[k]) ** 2) / (2 * r2))
            if n == k:
                Rn[n, k] = Rn[n, k] + s2 ** 2
    k = Rn[-1, :N - 1]
    d = Rn[-1, -1]
    Rn_sub_inv = np.linalg.inv(Rn[:N - 1, :N - 1])
    y_hat = k.T @ Rn_sub_inv @ closer_y
    variance = d - k.T @ Rn_sub_inv @ k

In [ ]:
# to plot and see a covariance matrix
plt.figure()
plt.matshow(np.abs(Rn), fignum=0)
plt.colorbar()
plt.title('$R_N$ covariance matrix')
plt.tight_layout()
# plt.savefig('./corr_coeff.png')
plt.show()

In [ ]:
# De-normalization, to make data readable by a medical doctor
y_hat_tr = y_hat_tr_norm * sy + my
y_tr = y_tr_norm * sy + my
y_hat_te = y_hat_te_norm * sy + my
y_te = y_te_norm * sy + my

In [ ]:
E_tr = y_tr - y_hat_tr
E_te = y_te - y_hat_te
e = [E_tr, E_te]
plt.figure(figsize=(6, 4))
plt.hist(e, bins=40, density=True, histtype='bar')
label = ['Training', 'Test']
plt.xlabel(r'$e=y-\^y$')
plt.ylabel(r'$P(e$ in bin$)$')
plt.legend(label)
plt.ylim([0, 0.21])
plt.xlim([-11, 45])
plt.grid()
plt.title('LLS - Error histograms')
plt.tight_layout()
plt.savefig('./LLS-Error-hist.png')
plt.show()

The last plot shows that the error pdf is like a mixture of two gaussian pdfs.
The fact that the two pdfs of training and test looks very similar is because there's no overfitting. If overfitting
was present, the error in the training would have been more concentrated on the 0 value, while the pdf of the test
would have depended on the similarity of the two datasets: if similar, the two pdfs could have been similar, otherwise
the test one would have quite even distributed values, in height, from the regression line, enlighting a randomness on the performance in guessing the new values. (Discuss this)

In [ ]:
y_hat_te = (X_te_norm @ w_hat) * sy + my
y_te = y_te_norm * sy + my
plt.figure(figsize=(6, 4))
plt.plot(y_te, y_hat_te, '.')
v = plt.axis()
plt.plot([v[0], v[1]], [v[0], v[1]], 'r', linewidth=2)
plt.xlabel(r'$y$')
plt.ylabel(r'$\^y$')
plt.grid()
plt.title('LLS - Test')
plt.tight_layout()
plt.savefig('./LLS-yhat_vs_y.png')
plt.show()

The estimation is quite reliable, except for some cases that stay near 7-8 UPDRS points far from the
original value. This justifies the second gaussian with its average near 7-8. 

In [ ]:
E_tr_min = E_tr.min()
E_tr_max = E_tr.max()
E_tr_mu = E_tr.mean()
E_tr_sig = E_tr.std()
E_tr_MSE = np.mean(E_tr ** 2)
R2_tr = 1 - E_tr_MSE / np.var(y_tr)
c_tr = np.mean((y_tr - y_tr.mean()) * (y_hat_tr - y_hat_tr.mean())) / (y_tr.std() * y_hat_tr.std())
E_te_min = E_te.min()
E_te_max = E_te.max()
E_te_mu = E_te.mean()
E_te_sig = E_te.std()
E_te_MSE = np.mean(E_te ** 2)
R2_te = 1 - E_te_MSE / np.var(y_te)
c_te = np.mean((y_te - y_te.mean()) * (y_hat_te - y_hat_te.mean())) / (y_te.std() * y_hat_te.std())

In [ ]:
rows = ['Training', 'Test']
cols = ['min', 'max', 'mean', 'std', 'MSE', 'R^2', 'correlation coefficient']
p = np.array([
    [E_tr_min, E_tr_max, E_tr_mu, E_tr_sig, E_tr_MSE, R2_tr, c_tr],
    [E_te_min, E_te_max, E_te_mu, E_te_sig, E_te_MSE, R2_te, c_te]
])
results = pd.DataFrame(p, columns=cols, index=rows)
print(results)

Comments on these results:
The test dataset has one outlier, so it shows a relative high error.
The mean is close to zero for the training set (which is expected), while it is a bit higher for the test set because the mean of total_UPDRS is calculated on the train set, so since values are different it is a normal and possible scenario.
The $R^2$ (coefficient of determination, the "score of prediction") is 0.92, which is a good result for the regression.